In [2]:
# Imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import loginInfo
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import time
from find_inventory_element_and_click import find_inventory_element_and_click
from find_inventory_elements import find_inventory_elements
from wait_and_type_element import wait_and_type_element
from wait_and_click_element import wait_and_click_element
from click_menu_item import click_menu_item
from wait_for_element_by_text import wait_for_element_by_text
from get_integer_from_xpath import get_integer_from_xpath

In [3]:
# Open instance 
driver = webdriver.Chrome()
driver.get("https://ironwoodrpg.com")

# Sign into the website and go to the main page
wait_and_click_element(driver, '/html/body/app-component/div/div/div/auth-page/div[4]/button[2]' , timeout=100)
wait_and_click_element(driver, '/html/body/app-component/div/div/div/auth-page/modal-component/div/div[2]/div/div/form/div[4]/button' , timeout=100)
wait_and_type_element(driver, '/html/body/app-component/div/div/div/auth-page/modal-component/div/div[2]/div/div/form/div[1]/input', loginInfo.usernamelist[0], timeout=100)
wait_and_type_element(driver, '/html/body/app-component/div/div/div/auth-page/modal-component/div/div[2]/div/div/form/div[3]/input', loginInfo.passwordlist[0], timeout=100)
wait_and_click_element(driver, '/html/body/app-component/div/div/div/auth-page/modal-component/div/div[2]/div/div/form/button' , timeout=100)


In [5]:
# Tutorial Page One
click_menu_item(driver, "Tutorial")
click_menu_item(driver, "Gather a Pine Log from Woodcutting")
click_menu_item(driver, "Pine Log") # Fails a lot of the time but needed incase Pine Log is not selected already
wait_and_click_element(driver, "/html/body/app-component/div/div/div/skill-page/div/div[1]/button") # "Gather" defaults to gather potion so XPath to gather must be used
time.sleep(10) # Action only takes 4.1 but accounting for bad connection
click_menu_item(driver, "Stop & Loot")


click_menu_item(driver, "Tutorial")
click_menu_item(driver, "Gather a Copper Ore from Mining")
click_menu_item(driver, "Copper Rock")
wait_and_click_element(driver, "/html/body/app-component/div/div/div/skill-page/div/div[1]/button") # "Gather" defaults to gather potion so XPath to gather must be used
time.sleep(10) # Action only takes 4.1 but accounting for bad connection
click_menu_item(driver, "Stop & Loot")


click_menu_item(driver, "Tutorial")
click_menu_item(driver, "Create Charcoal from Smelting by using the")
click_menu_item(driver, "Charcoal")
click_menu_item(driver, "Pine Log")
click_menu_item(driver, "Max")
click_menu_item(driver, "Create")


click_menu_item(driver, "Tutorial")
click_menu_item(driver, "Craft a Copper Bar from Smelting")
wait_and_click_element(driver, "/html/body/app-component/div/div/div/skill-page/div/div[1]/div[3]/button[2]") # "Craft" defaults to craft potion so XPath to Craft mut be used
time.sleep(10) # Action only takes 4.1 but accounting for bad connection

click_menu_item(driver, "Tutorial")
click_menu_item(driver, "Claim")

Attempt 1 of 3 to click 'Tutorial'
Successfully clicked on 'Tutorial'
Attempt 1 of 3 to click 'Gather a Pine Log from Woodcutting'
Successfully clicked on 'Gather a Pine Log from Woodcutting'
Attempt 1 of 3 to click 'Pine Log'
Could not find clickable button with text 'Pine Log' on attempt 1
Attempt 2 of 3 to click 'Pine Log'
Could not find clickable button with text 'Pine Log' on attempt 2
Attempt 3 of 3 to click 'Pine Log'
Could not find clickable button with text 'Pine Log' on attempt 3
Failed to click on 'Pine Log' after 3 attempts
Attempt 1 of 3 to click 'Stop & Loot'
Successfully clicked on 'Stop & Loot'
Attempt 1 of 3 to click 'Tutorial'
Successfully clicked on 'Tutorial'
Attempt 1 of 3 to click 'Gather a Copper Ore from Mining'
Successfully clicked on 'Gather a Copper Ore from Mining'
Attempt 1 of 3 to click 'Copper Rock'
Successfully clicked on 'Copper Rock'
Attempt 1 of 3 to click 'Stop & Loot'
Successfully clicked on 'Stop & Loot'
Attempt 1 of 3 to click 'Tutorial'
Successfu

True

In [20]:
# Tutorial Page Two
click_menu_item(driver, "Tutorial")
click_menu_item(driver, "Craft a Copper Sword from Forging")
click_menu_item(driver, "Copper")
click_menu_item(driver, "Copper Sword")
wait_and_click_element(driver, "/html/body/app-component/div/div/div/skill-page/div/div[1]/div[3]/button[2]") # "Craft" defaults to craft potion so XPath to Craft mut be used
time.sleep(10) # Action only takes 4.1 but accounting for bad connection

click_menu_item(driver, "Tutorial")
click_menu_item(driver, "Craft a Copper Shield from Smithing")
click_menu_item(driver, "Copper")
click_menu_item(driver, "Copper Shield")
wait_and_click_element(driver, "/html/body/app-component/div/div/div/skill-page/div/div[1]/div[3]/button[2]") # "Craft" defaults to craft potion so XPath to Craft mut be used
time.sleep(10) # Action only takes 4.1 but accounting for bad connection
click_menu_item(driver, "Stop & Loot") # Need to stop and loot this time so shield is available to equip

click_menu_item(driver, "Tutorial")
click_menu_item(driver, "Go to the Equipment page")
click_menu_item(driver, "Main Hand")
click_menu_item(driver, "Copper Sword")
click_menu_item(driver, "Off Hand")
click_menu_item(driver, "Copper Shield")

click_menu_item(driver, "Tutorial")
click_menu_item(driver, "Claim")


Attempt 1 of 3 to click 'Tutorial'
Successfully clicked on 'Tutorial'
Attempt 1 of 3 to click 'Craft a Copper Sword from Forging'
Successfully clicked on 'Craft a Copper Sword from Forging'
Attempt 1 of 3 to click 'Copper'
Successfully clicked on 'Copper'
Attempt 1 of 3 to click 'Copper Sword'
Successfully clicked on 'Copper Sword'
Attempt 1 of 3 to click 'Tutorial'
Successfully clicked on 'Tutorial'
Attempt 1 of 3 to click 'Craft a Copper Shield from Smithing'
Successfully clicked on 'Craft a Copper Shield from Smithing'
Attempt 1 of 3 to click 'Copper'
Successfully clicked on 'Copper'
Attempt 1 of 3 to click 'Copper Shield'
Successfully clicked on 'Copper Shield'
Attempt 1 of 3 to click 'Stop & Loot'
Successfully clicked on 'Stop & Loot'
Attempt 1 of 3 to click 'Tutorial'
Successfully clicked on 'Tutorial'
Attempt 1 of 3 to click 'Go to the Equipment page'
Successfully clicked on 'Go to the Equipment page'
Attempt 1 of 3 to click 'Main Hand'
Successfully clicked on 'Main Hand'
Attemp

True

In [5]:
# Tutorial Page Three
# click_menu_item(driver, "")
click_menu_item(driver, "Tutorial")
click_menu_item(driver, "Craft 150 Copper Helmets from Smithing")
wait_and_click_element(driver, "/html/body/app-component/div/div/div/skill-page/div/div[1]/div[3]/button[2]") # "Craft" defaults to craft potion so XPath to Craft mut be used
numHelmet = 0
while numHelmet < 51:
    numHelmet = get_integer_from_xpath(driver, "/html/body/app-component/div/div/div/skill-page/div/div[1]/div[3]/div[2]/div[3]")
    time.sleep(1)
click_menu_item(driver, "Stop & Loot") # Need to stop and loot this time so bars are not wasted

click_menu_item(driver, "Tutorial")
click_menu_item(driver, "Go to the Inventory page")
click_menu_item(driver, "Upgrade")
find_inventory_element_and_click(driver, "helmet")
wait_and_click_element(driver, "/html/body/app-component/div/div/div/inventory-page/modal-component/div/div[2]/div/div/div[7]/div")
wait_and_click_element(driver, "/html/body/app-component/div/div/div/inventory-page/modal-component/div/div[2]/div/div/div[7]/div")
wait_and_click_element(driver, "/html/body/app-component/div/div/div/inventory-page/modal-component/div/div[2]/div/div/div[7]/div")
click_menu_item(driver, "Tutorial")
click_menu_item(driver, "Claim")


Attempt 1 of 3 to click 'Tutorial'
Successfully clicked on 'Tutorial'
Attempt 1 of 3 to click 'Craft 150 Copper Helmets from Smithing'
Successfully clicked on 'Craft 150 Copper Helmets from Smithing'
Attempt 1 of 3 to click 'Stop & Loot'
Successfully clicked on 'Stop & Loot'
Attempt 1 of 3 to click 'Tutorial'
Successfully clicked on 'Tutorial'
Attempt 1 of 3 to click 'Go to the Inventory page'
Successfully clicked on 'Go to the Inventory page'
Attempt 1 of 3 to click 'Upgrade'
Successfully clicked on 'Upgrade'
Button found: <button _ngcontent-cmm-c44="" type="button" class="item ng-star-inserted"><div _ngcontent-cmm-c44="" class="item-background"><div _ngcontent-cmm-c44="" class="image"><img _ngcontent-cmm-c44="" src="/assets/items/armor-copper-helmet.png"><div _ngcontent-cmm-c44="" class="stars"><!----></div></div><div _ngcontent-cmm-c44="" class="amount">8</div></div></button>
Successfully clicked button containing image: helmet


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7C282EEA2+31554]
	(No symbol) [0x00007FF7C27A7ED9]
	(No symbol) [0x00007FF7C266872A]
	(No symbol) [0x00007FF7C26B8434]
	(No symbol) [0x00007FF7C26B853C]
	(No symbol) [0x00007FF7C26FF6A7]
	(No symbol) [0x00007FF7C26DD06F]
	(No symbol) [0x00007FF7C26FC977]
	(No symbol) [0x00007FF7C26DCDD3]
	(No symbol) [0x00007FF7C26AA33B]
	(No symbol) [0x00007FF7C26AAED1]
	GetHandleVerifier [0x00007FF7C2B38B1D+3217341]
	GetHandleVerifier [0x00007FF7C2B85AE3+3532675]
	GetHandleVerifier [0x00007FF7C2B7B0E0+3489152]
	GetHandleVerifier [0x00007FF7C28DE776+750614]
	(No symbol) [0x00007FF7C27B375F]
	(No symbol) [0x00007FF7C27AEB14]
	(No symbol) [0x00007FF7C27AECA2]
	(No symbol) [0x00007FF7C279E16F]
	BaseThreadInitThunk [0x00007FFE5F13257D+29]
	RtlUserThreadStart [0x00007FFE5FD6AF28+40]


In [8]:
wait_and_click_element(driver, "/html/body/app-component/div/div/div/inventory-page/modal-component/div/div[2]/div/div/div[7]/div")



In [12]:
import subprocess

def kill_chrome():
    # run the 'pkill' command to kill all 'chrome' processes
    subprocess.run(['pkill', 'Google Chrome'])

kill_chrome()

FileNotFoundError: [WinError 2] The system cannot find the file specified